In [13]:
import poppy
import jax
import jax.numpy as jnp
import functools
import galois

In [2]:
p = 6661
n = 3
f = poppy.field( p, n )
f.X, f.INV

(Array([[[   1,    0,    0],
         [   0,    1,    0],
         [   0,    0,    1]],
 
        [[   0,    0,    6],
         [   1,    0, 6660],
         [   0,    1,    0]],
 
        [[   0,    6,    0],
         [   0, 6660,    6],
         [   1,    0, 6660]]], dtype=int64),
 Array([   0,    1, 3331, ..., 2220, 3330, 6660], dtype=int64))

In [3]:
d = 4
a = poppy.random( ( d, d ), f, poppy.SEED + 0 )
b = poppy.random( ( d, d ), f, poppy.SEED + 1 )

In [4]:
a.shape, a.lift.shape, a.lift

((4, 4),
 (12, 12),
 Array([[2616, 2367, 6596, 6247, 2613, 5145, 1013, 6059, 2241, 4296,  121,
         4319],
        [5540, 5552, 3488, 4188, 2481, 5086, 3704, 5554, 2355, 1830, 5386,
         4952],
        [3725, 5540, 5552, 3766, 4188, 2481, 2120, 3704, 5554, 5571, 1830,
         5386],
        [  58, 1188, 1218,  853, 4131, 5209,  226, 6044, 2986, 2265, 5039,
         1977],
        [ 203, 6521,  985, 6419, 3495, 4373, 2718, 1439, 3326, 3660,  315,
         1379],
        [ 198,  203, 6521, 4019, 6419, 3495, 5448, 2718, 1439, 1950, 3660,
          315],
        [3283, 4717, 5799, 5348, 5143, 5739, 3772, 1840, 4869, 3021, 3050,
         3429],
        [4297, 3607,  420, 4287, 5601,  856, 4142, 1245, 4359, 3902, 4733,
         5809],
        [6337, 4297, 3607, 6408, 4287, 5601, 2527, 4142, 1245, 4949, 3902,
         4733],
        [3615, 5997, 5278, 3627, 3415, 6485, 3376, 5794, 2866, 4079, 5540,
         4475],
        [3100, 5946, 2897, 2191, 4168, 1224, 2698,  190, 3096, 1856, 5

In [5]:
ai = a.inv( )
bi = b.inv( )

In [6]:
print( a.lift[ 0, : 10 ] )
( a - a @ ai @ a @ bi @ b ).lift

[2616 2367 6596 6247 2613 5145 1013 6059 2241 4296]


Array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [7]:
N  = 10000
d  = 2
p  = 7
p1 = 13
p2 = 17
n  = 6
q  = p ** n
f  = poppy.field( p,  n )
F  = poppy.field( p1, d )
FF = poppy.field( p2, n )
a  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 0 ), ( N, F.n, F.n ), 0,       q, dtype = jnp.int64 )
AA  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ),  ( F.n * FF.n, F.n * FF.n ), 0, 2 ** 32, dtype = jnp.int64 )
BB  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 2 ),  ( F.n * FF.n, F.n * FF.n ), 0, 2 ** 32, dtype = jnp.int64 )
A  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ),  ( FF.n, FF.n ), 0, 2 ** 32, dtype = jnp.int64 )
B  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 2 ),  ( FF.n, FF.n ), 0, 2 ** 32, dtype = jnp.int64 )

@jax.jit
def poppyhash( a ):
    b = poppy.proj( AA @ a + BB, F )
    c = poppy.proj( A @ b + B, FF )
    return c % ( 4 * N )

liftbatch = jax.vmap( poppy.lift, in_axes = ( 0, None ) )
hashbatch = jax.vmap( poppyhash )

ap = liftbatch( a, f )
h  = hashbatch( ap ).squeeze( )
au = jnp.unique( a, axis = 0 )
hu = jnp.unique( h )
len( au ), len( hu ), F.q, FF.q

(10000, 17, 169, 24137569)

In [12]:
f = poppy.field( 7, 3 )
a  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 0 ), ( 2, 2 ), 0, f.p, dtype = jnp.int64 )
a1 = poppy.array( a[ 0, 0 ], f )
a2 = poppy.array( a[ 0, 1 ], f )
a3 = poppy.array( a[ 1, 0 ], f )
a4 = poppy.array( a[ 1, 1 ], f )
a, a1.lift, a4.lift, (a[ 0, 0 ] + a[ 1, 1])%f.q

(Array([[4, 1],
        [5, 5]], dtype=int64),
 Array([[4, 0, 0],
        [0, 4, 0],
        [0, 0, 4]], dtype=int64),
 Array([[5, 0, 0],
        [0, 5, 0],
        [0, 0, 5]], dtype=int64),
 Array(9, dtype=int64))

In [14]:
GF = galois.GF( 7 ** 3 )
GF(4)+GF(5)

GF(2, order=7^3)